In [124]:
import fonctions


In [125]:
from math import *
rho = 6.371*10**6
def convertirLat(phi1, phi2):
    return rho*2*pi/360*(phi1-phi2)

def convertirLon(phi1, phi2):
    return rho*cos(2*pi*2.34842/360)*2*pi/360*(phi1-phi2)



In [126]:
import pandas as pd
instance = pd.read_csv("instances/instance_01.csv")

vehicules = pd.read_csv("vehicles.csv")
def reference_travel_time(f, i, j):
    x1 = instance.loc[i]["longitude"]
    x2 = instance.loc[j]["longitude"]
    y1 = instance.loc[i]["latitude"]
    y2 = instance.loc[j]["latitude"]
    sf = vehicules.loc[f]["speed"]
    pf = vehicules.loc[f]["parking_time"]
    return fonctions.distanceE((x1,y1), (x2,y2))/sf + pf

In [127]:
def miam(f,t):
    k = 0 
    a = [vehicules.loc[f]["fourier_cos_0"],vehicules.loc[f]["fourier_cos_1"],vehicules.loc[f]["fourier_cos_2"],vehicules.loc[f]["fourier_cos_3"]]
    b = [vehicules.loc[f]["fourier_sin_0"],vehicules.loc[f]["fourier_sin_1"],vehicules.loc[f]["fourier_sin_2"],vehicules.loc[f]["fourier_sin_3"]]
    for i in range(4):
        k+= a[i]*cos(i*2*pi/86400*t) + b[i]*sin(i*2*pi/86400 * t)
    return k

In [128]:
def travel_time(f, i, j, t):
    return reference_travel_time(f, i, j)*miam(f,t)

In [129]:
def rental_cost(route):
    return vehicules.loc[route[0]-1]["rental_cost"]


In [130]:
rental_cost([1,2])

260.0

In [131]:
def fuel_cost(route):
    cout = vehicules.loc[route[0]-1]["fuel_cost"]
    chemin = route[2]
    k=0
    for i in range(len(chemin)-1):
        x1 = instance.loc[chemin[i],"longitude"]
        x2 = instance.loc[chemin[i+1],"longitude"]
        y1 = instance.loc[chemin[i],"latitude"]
        y2 = instance.loc[chemin[i+1],"latitude"]
        k+=fonctions.distanceM((x1,y1),(x2,y2))
    
    return cout*k


In [132]:
def radius_cost(route):
    cout = vehicules.loc[route[0]-1]["radius_cost"]
    chemin = route[2]
    maxi = 0
    for i in range(1,len(chemin)-1):
        for j in range(1, len(chemin)-1):
            x1 = instance.loc[chemin[i],"longitude"]
            x2 = instance.loc[chemin[i+1],"longitude"]
            y1 = instance.loc[chemin[i],"latitude"]
            y2 = instance.loc[chemin[i+1],"latitude"]
            dist=fonctions.distanceE((x1,y1),(x2,y2))
            if dist>maxi:
                maxi = dist

    return cout*(0.5*maxi)

In [133]:
def cout_total(sol):
    total = 0 
    for i in sol:
        total += rental_cost(i)+ fuel_cost(i) + radius_cost(i)
    return total

In [134]:

import numpy as np
def arrivees_departs(f, chemin):
    n = len(chemin)
    garer = vehicules.loc[f-1]["parking_time"]
    arrivees = np.zeros(n)
    departs = np.zeros(n)
    t = 0 
    for i in range(1,n-1):
        arrivees[i] = departs[i-1] + travel_time(f, chemin[i-1], chemin[i], t)
        departs[i] = max(arrivees[i]+garer, instance.loc[i]["window_start"])+instance.loc[i]["delivery_duration"]
        t = departs[i]
    arrivees[n-1] = departs[n-1] + travel_time(f, chemin[n-2], chemin[n-1], t)
    return arrivees[1:], departs[:-1]

In [135]:
arrivees_departs(1, [0,1,2])

(array([1324.52961213,  833.818266  ]), array([    0., 24900.]))

In [136]:
instance.loc[0]["window_start"]

nan

In [137]:
cout_total([[1,1,[0,1,2,0]],[2,1,[0,3,4,0]]])

732.8558968587195

In [138]:
shape = instance.shape
nb_routes = shape[0]
print(nb_routes)

def construction_solution_initiale () :
    solution_initiale = []
    for j in range (1,nb_routes) :
        # un sommet i
        l_i = instance.loc[j, "delivery_duration"] #delivery duration
        a_1 = reference_travel_time(0, 0, j)
        d_1 = a_1 + l_i
        solution_initiale.append([1, 1, [0,j,0]])
    return solution_initiale
x = construction_solution_initiale()



17


In [139]:
max_2eme = max([sous_liste[1] for sous_liste in x])
max_2eme

1

In [140]:
import csv

def creer_csv_colonnes(x, nom_fichier="donnees.csv"):
    
    # Noms des 20 colonnes
    colonnes = ["family"]+[f"order_{i+1}" for i in range(max([sous_liste[1] for sous_liste in x]))]


    with open(nom_fichier, mode='w', newline='', encoding='utf-8') as fichier:
        writer = csv.writer(fichier)
        writer.writerow(colonnes)  # écrit l’en-tête


        for j in x:
            ligne = [j[0]]+j[2][1:-1]
            writer.writerow(ligne)

    print(f"Fichier '{nom_fichier}' créé ")

# Exemple d'utilisation
creer_csv_colonnes(x,"exemple.csv")

Fichier 'exemple.csv' créé 


In [141]:
def constraint_3a (solution) : #A vehicle can only depart from an order after having arrived and then performed the delivery:
    for r in solution : 
        for i in range (1, len(r[2])-1) :
            if r[3][i] + instance.loc[i, "delivery_duration"] - r[4][i] > 0 : 
                return False 
    return True 

def constraint_3b (solution) : #A vehicle can only arrive at its first vertex after having departed and traveled from the depot:
    for r in solution : 
        if r[3][0] < r[4][0] + reference_travel_time(r[0], 0, r[2][1], 0) : 
            return False 
    return True
        

def constraint_3c (solution) : #A vehicle can only arrive at a subsequent vertex after having departed and traveled from the previous vertex:      
    for r in solution : 
        for i in range (1, len(r[2])) : 
            if r[3][i] < r[4][i-1] + reference_travel_time(r[0], 0, r[2][1], r[4][i-1]):
                return False
    return True 

def constraint_3d (solution) : #Arrival times at each order must fall within the order’s delivery window:
    for r in solution : 
        for i in range (1, len(r[2])-1) : 
            t_min_i = instance.loc[i, "window_start"]
            t_max_i = instance.loc[i, "window_end"]
            if r[3][i]<t_min_i or r[3][i]>t_max_i : 
                return False 
    return True 

In [142]:
def cout_chemin(t):
    return rental_cost(t)+ fuel_cost(t) + radius_cost(t)

In [143]:
from math import *

rho = 6.371*10**6
def convertirLat(phi1, phi2):
    return rho*2*pi/360*(phi1-phi2)

def convertirLon(phi1, phi2):
    return rho*cos(2*pi*2.34842/360)*2*pi/360*(phi1-phi2)

def distanceM(point1, point2):
    return abs(convertirLon(point1[0], point2[0]))+ abs(convertirLat(point1[1], point2[1]))

def distanceE(point1, point2):
    return sqrt((convertirLon(point1[0], point2[0]))**2+ (convertirLat(point1[1], point2[1]))**2)



In [144]:
def chemin_cher(x): #que des routes de longueur 1 sommet
    for i in x:
        print(cout_chemin(i))

chemin_cher(x)

274.0861744142604
277.92506520765585
275.1212328914292
303.0548128630112
270.3833630892687
275.71082550962035
279.80910851386216
271.49407451229916
271.2409210400517
278.71561127589086
276.996487697461
278.321280671698
285.6388705927677
289.3526440425084
273.60874529336866
279.9632971448403


In [145]:
fuel_cost([1,1,[0,4,0]])

32.026219198501636

In [146]:
for i in range(1,16):
    print(distanceE((instance.loc[0, "longitude"], instance.loc[0,"latitude"]),(instance.loc[i, "longitude"], instance.loc[i, "latitude"])))

9006.999373845254
11700.24261262125
9938.160584040399
22726.828301226647
6307.862305689231
10285.74734733998
11429.748155095485
7354.1108776058245
6821.597288303815
10857.716305523096
10686.261645380346
10961.776611716343
13543.20197616006
15453.550421956688
8853.556177465522


In [147]:
def chemins_ordonnes(x):
    return sorted(x,key =  lambda t :cout_chemin(t), reverse=True)


In [148]:
chemins_ordonnes(x)

[[1, 1, [0, 4, 0]],
 [1, 1, [0, 14, 0]],
 [1, 1, [0, 13, 0]],
 [1, 1, [0, 16, 0]],
 [1, 1, [0, 7, 0]],
 [1, 1, [0, 10, 0]],
 [1, 1, [0, 12, 0]],
 [1, 1, [0, 2, 0]],
 [1, 1, [0, 11, 0]],
 [1, 1, [0, 6, 0]],
 [1, 1, [0, 3, 0]],
 [1, 1, [0, 1, 0]],
 [1, 1, [0, 15, 0]],
 [1, 1, [0, 8, 0]],
 [1, 1, [0, 9, 0]],
 [1, 1, [0, 5, 0]]]

In [149]:
import itertools
def constraint_3a_permut (liste, arrivee, depart) : #A vehicle can only depart from an order after having arrived and then performed the delivery: 
    for i in range (1, len(liste)) :
        if arrivee[i-1] + instance.loc[i, "delivery_duration"] > depart[i] : 
            return False 
    return True

def constraint_3b_permut (liste, arrivee, depart) : #A vehicle can only arrive at its first vertex after having departed and traveled from the depot:
    if arrivee[0] < depart[0] + travel_time(0, liste[0], liste[1], 0) : 
        return False 
    return True

def constraint_3c_permut (liste, arrivee, depart) : #A vehicle can only arrive at a subsequent vertex after having departed and traveled from the previous vertex:      
    for i in range (1, len(liste)) : 
        if arrivee[i] < depart[i] + travel_time(0, liste[i-1], liste[i], depart[i]):
            return False
    return True

def constraint_3d_permut(liste, arrivee, depart) : #Arrival times at each order must fall within the order’s delivery window: 
    for i in range (1, len(liste)-1) : 
        t_min_i = instance.loc[i, "window_start"]
        t_max_i = instance.loc[i, "window_end"]
        if arrivee[i-1]<t_min_i or depart[i]>t_max_i : 
            return False 
    return True 








def permutations_faisables(T) :
    permutations = list(itertools.permutations(T))
    possibles = []
    for l in permutations : 
        arrivee, depart = arrivees_departs(1,[0]+list(l)+[0])
        if constraint_3a_permut (l, arrivee, depart) and constraint_3b_permut (l, arrivee, depart) and constraint_3c_permut (l, arrivee, depart) and constraint_3d_permut (l, arrivee, depart) :
            possibles.append(l)
    return possibles 

In [150]:
permutations = list(itertools.permutations([1,2]))
list(permutations[0])

arrivees_departs(1, [0]+list(permutations[0])+[0])



(array([ 1324.52961213, 25733.818266  ,  1725.02116774]),
 array([    0.      , 24900.      , 26433.818266]))

In [151]:
permutations_faisables([1,2])

[(1, 2), (2, 1)]

In [152]:
def permutations_ordonnées(l):
    rer = permutations_faisables(l)
    return sorted(rer, key =  lambda t :cout_chemin([1,1,t]), reverse=True)

In [153]:
permutations_ordonnées([1,2])

[(1, 2), (2, 1)]

In [ ]:
P = chemins_ordonnes(x)
t = P[0]
cout_eco=0
chemin_elim=None
caca = t[2][1:-1]
permu = None

for i in range(1,len(P[1:])):
    cout_init = cout_chemin(t) + cout_chemin(P[i])
    y = P[i][2][1:-1]
    coout = cout_chemin([1,1,[0]+list(permutations_ordonnées(caca+y)[0])+[0]])
    if cout_init-coout >cout_eco:
        cout_eco = cout_init-coout
        chemin_elim=P[i]
        permu = list(permutations_ordonnées(caca+y)[0])

x[x.index(t)] = [1,1, [0]+permu+[0]]
x.pop(x.index(chemin_elim))


[1, 1, [0, 14, 0]]

In [172]:
x

[[1, 1, [0, 1, 0]],
 [1, 1, [0, 2, 0]],
 [1, 1, [0, 3, 0]],
 [1, 1, [0, 4, 14, 0]],
 [1, 1, [0, 5, 0]],
 [1, 1, [0, 6, 0]],
 [1, 1, [0, 7, 0]],
 [1, 1, [0, 8, 0]],
 [1, 1, [0, 9, 0]],
 [1, 1, [0, 10, 0]],
 [1, 1, [0, 11, 0]],
 [1, 1, [0, 12, 0]],
 [1, 1, [0, 13, 0]],
 [1, 1, [0, 15, 0]],
 [1, 1, [0, 16, 0]]]

In [161]:
P = chemins_ordonnes(x)
t = P[0]

cout_eco = 0
chemin_elim = None

# sous-liste du chemin t sans les bornes 0
caca = t[2][1:-1]

for i, autre in enumerate(P[1:], start=1):
    cout_init = cout_chemin(t) + cout_chemin(autre)

    y = autre[2][1:-1]  # même logique : on enlève les bornes
    fusion = [0] + list(permutations_ordonnées(caca + y)[0]) + [0]

    cout_fusion = cout_chemin([1, 1, fusion])

    gain = cout_init - cout_fusion
    if gain > cout_eco:
        cout_eco = gain
        chemin_elim = i

P[0]

[1, 1, [0, 4, 0]]